In [ ]:
import numpy as np, pandas as pd, scipy.stats as st

data = pd.read_csv('data.csv')
num_obvs = 30_000
# risks per sector
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
#sec_loading maps sector to its risk, t is the threshold for defaults
data['sec_loading'], data['t'] = r[data['sector'].values], st.norm.ppf(data.p)
# 100k monte carlo simulations and len(r)+len(data) risk factors per one sample
num_dimensions = len(r) + len(data)
from scipy.stats import qmc

sobol_sampler = qmc.Sobol(d=num_dimensions, scramble=True)
quasi_random_samples = sobol_sampler.random_base2(m=int(np.log2(num_obvs)))
from scipy.stats import norm

#factors = st.norm.ppf(quasi_random_samples)
shift = 2.0  # Shift parameter to bias the sampling distribution towards the tail

# Simulate factors and initialize
factors = np.random.normal(shift, 1, (100_000, len(r) + len(data)))  # Shifted Gaussian

def process_obs(obs):
    m_factor, sec_factor, res_factor = obs[0], obs[:len(r)][data.sector.values], obs[len(r):]

    # Adjusted loss indicator threshold
    ind = np.sqrt(r[0]) * m_factor + np.sqrt(data.sec_loading - r[0]) * sec_factor \
          + np.sqrt(1 - data.sec_loading) * res_factor < (data.t - shift)

    # Compute losses
    loss = np.zeros((len(data),))
    loss[ind] = data[ind].m + data[ind].d * np.random.standard_t(3, size=sum(ind))

    # Compute weight for importance sampling correction
    weight = np.exp(-shift * obs[0] + 0.5 * shift**2)
    return np.sum(loss) * weight, weight

from joblib import Parallel, delayed
import time
from scipy.stats import t

sample = []
sobol_student = qmc.Sobol(d=1, scramble=True)

answers = Parallel(n_jobs=-1)(delayed(process_obs)(obs) for obs in factors)
answers = np.array(answers)

In [ ]:
samples = answers[:, 0]
print(len(samples))

In [ ]:
weights = answers[:, 1]
print(len(weights))

In [ ]:
# Calculate VaR using weighted losses
weighted_losses = -np.array(samples)
sorted_losses = np.sort(weighted_losses)
VaR = sorted_losses[int(0.999 * len(sorted_losses))]  # Empirical 99.9% quantile
print(f"VaR with Importance Sampling: {VaR}")

In [ ]:
sorted_samples = np.argsort(samples)
sorted_weights = np.argsort(weights)
cumulative_weights = np.cumsum(sorted_samples) / np.sum(sorted_weights)

In [ ]:
VaR_index = np.searchsorted(cumulative_weights, 0.0001)
VaR = sorted_samples[VaR_index]

In [ ]:
sorted_samples[:100]

In [ ]:
VaR = sorted([-s for s in sample])[int(0.01 * num_obvs)]
VaR

In [ ]:
sample

ANTH

In [2]:
import numpy as np, pandas as pd, scipy.stats as st

data = pd.read_csv('data.csv')
num_obvs = 30_000
# risks per sector
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
#sec_loading maps sector to its risk, t is the threshold for defaults
sec_loading, datat = r[data['sector'].values], st.norm.ppf(data.p)


In [3]:
import numpy as np, pandas as pd
from scipy.stats import t

data = pd.read_csv('data.csv')

num_obvs = 30_000
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
factors, sample = np.random.normal(0,1, (num_obvs, len(r)+len(data))), []

antithetic_factors = -factors

factors = np.vstack([factors, antithetic_factors])
sample_antithetic = []
def process_obs(obs):
    m_factor = obs[0]
    sec_factor = obs[:len(r)][data.sector.values]
    res_factor = obs[len(r):]

    ind = (
                  r[0]**0.5 * m_factor
                  + (sec_loading - r[0])**0.5 * sec_factor
                  + (1 - sec_loading)**0.5 * res_factor
          ) < datat
    loss = np.zeros((len(data),))
    loss[ind] = data[ind].m + data[ind].d * t.rvs(df=3,size=sum(ind))
    return sum(loss), np.var(loss)

In [4]:

from joblib import Parallel, delayed
import time
from scipy.stats import t

answers = Parallel(n_jobs=-1)(delayed(process_obs)(obs) for obs in factors)
answers = np.array(answers)

In [5]:
sample_antithetic = answers[:, 0]
vars = answers[:, 1]
Var_antithetic = sorted([-s for s in sample_antithetic])[int(0.001*num_obvs)]
Var_antithetic

-21825.159480973656

In [6]:
np.mean(vars)

1342.5501061476914

NEW

In [7]:
import numpy as np
import pandas as pd
from scipy.stats import t, norm
from joblib import Parallel, delayed

# Wczytanie danych
data = pd.read_csv('data.csv')

# Parametry symulacji
num_obvs = 30_000
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
sector_indices = data['sector'].values
sec_loading = r[sector_indices]
datat = norm.ppf(data['p'])

# Parametry do stratyfikacji
num_strata = 10
strata_bounds = np.linspace(-3, 3, num_strata + 1)

# Funkcja generująca stratyfikowane czynniki
def stratified_factors(num_samples, dim, strata_bounds):
    factors = []
    samples_per_stratum = num_samples // len(strata_bounds)
    for i in range(len(strata_bounds) - 1):
        low, high = strata_bounds[i], strata_bounds[i + 1]
        stratum = np.random.uniform(low, high, (samples_per_stratum, dim))
        factors.append(stratum)
    return np.vstack(factors)

# Funkcja obliczająca straty
def process_obs(obs):
    m_factor = obs[0]  # Czynnik ogólny
    sec_factor = obs[:len(r)][sector_indices]  # Czynniki sektorowe
    res_factor = obs[len(r):]  # Czynniki resztowe

    # Zmienna kontrolna: przybliżenie straty jako wartość oczekiwana
    control_variate = (
            r[0]**0.5 * m_factor
            + (sec_loading - r[0])**0.5 * sec_factor
            + (1 - sec_loading)**0.5 * res_factor
    )

    # Ograniczanie wartości t-Studenta, aby zmniejszyć wpływ ekstremalnych strat
    ind = control_variate < datat
    loss = np.zeros((len(data),))
    if np.any(ind):
        loss[ind] = data.loc[ind, 'm'].values + data.loc[ind, 'd'].values * np.clip(t.rvs(df=3, size=sum(ind)), -5, 5)

    # Korekcja zmienną kontrolną
    E_control_variate = 0  # Oczekiwana wartość dla zmiennej kontrolnej
    c = np.cov(loss, control_variate)[0, 1] / np.var(control_variate) if np.var(control_variate) > 0 else 0
    corrected_loss = loss - c * (control_variate - E_control_variate)
    return np.sum(corrected_loss), np.var(corrected_loss)

# Generowanie stratyfikowanych próbek
factors = stratified_factors(num_obvs, len(r) + len(data), strata_bounds)
antithetic_factors = -factors
combined_factors = np.vstack([factors, antithetic_factors])

# Obliczenia w równoległych procesach
answers = Parallel(n_jobs=-1)(delayed(process_obs)(obs) for obs in combined_factors)
answers = np.array(answers)

# Wyniki
sample_antithetic = answers[:, 0]
vars = answers[:, 1]

In [8]:

# Symetryzacja wyników i redukcja wariancji
Var_antithetic = sorted(np.abs(sample_antithetic))[int(0.001*num_obvs)]

# Wyniki końcowe
print("Średnia wariancji:", np.mean(vars))
print("VaR antetyczny (100-ty wynik):", Var_antithetic)


Średnia wariancji: 58876.901880159356
VaR antetyczny (100-ty wynik): 0.0


In [9]:
import numpy as np
import pandas as pd
from scipy.stats import t, norm
from joblib import Parallel, delayed

# Wczytanie danych
data = pd.read_csv('data.csv')

# Parametry symulacji
num_obvs = 30_000
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
sector_indices = data['sector'].values
sec_loading = r[sector_indices]
datat = norm.ppf(data['p'])

# Parametry do stratyfikacji
num_strata = 10
strata_bounds = np.linspace(-3, 3, num_strata + 1)

# Funkcja generująca stratyfikowane czynniki
def stratified_factors(num_samples, dim, strata_bounds):
    factors = []
    samples_per_stratum = num_samples // len(strata_bounds)
    for i in range(len(strata_bounds) - 1):
        low, high = strata_bounds[i], strata_bounds[i + 1]
        stratum = np.random.uniform(low, high, (samples_per_stratum, dim))
        factors.append(stratum)
    return np.vstack(factors)

# Funkcja obliczająca straty
def process_obs(obs):
    m_factor = obs[0]  # Czynnik ogólny
    sec_factor = obs[:len(r)][sector_indices]  # Czynniki sektorowe
    res_factor = obs[len(r):]  # Czynniki resztowe

    # Zmienna kontrolna: przybliżenie straty jako wartość oczekiwana
    control_variate = (
            r[0]**0.5 * m_factor
            + (sec_loading - r[0])**0.5 * sec_factor
            + (1 - sec_loading)**0.5 * res_factor
    )

    # Ograniczanie wartości t-Studenta, aby zmniejszyć wpływ ekstremalnych strat
    ind = control_variate < datat
    loss = np.zeros((len(data),))
    if np.any(ind):
        loss[ind] = data.loc[ind, 'm'].values + data.loc[ind, 'd'].values * np.clip(t.rvs(df=3, size=sum(ind)), -5, 5)

    # Korekcja zmienną kontrolną
    E_control_variate = 0  # Oczekiwana wartość dla zmiennej kontrolnej
    c = np.cov(loss, control_variate)[0, 1] / np.var(control_variate) if np.var(control_variate) > 0 else 0
    corrected_loss = loss - c * (control_variate - E_control_variate)
    return np.sum(corrected_loss), np.var(corrected_loss)

# Generowanie stratyfikowanych próbek
factors = stratified_factors(num_obvs, len(r) + len(data), strata_bounds)
antithetic_factors = -factors
combined_factors = np.vstack([factors, antithetic_factors])

# Obliczenia w równoległych procesach
answers = Parallel(n_jobs=-1)(delayed(process_obs)(obs) for obs in combined_factors)
answers = np.array(answers)

# Wyniki
sample_antithetic = answers[:, 0]
vars = answers[:, 1]

In [10]:
import numpy as np
import pandas as pd
from scipy.stats import t, norm
from joblib import Parallel, delayed

# Wczytanie danych
data = pd.read_csv('data.csv')

# Parametry symulacji
num_obvs = 30_000
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
sector_indices = data['sector'].values
sec_loading = r[sector_indices]
datat = norm.ppf(data['p'])

# Funkcja obliczająca straty dla jednej obserwacji
def process_obs(obs):
    m_factor = obs[0]  # Czynnik ogólny
    sec_factor = obs[:len(r)][sector_indices]  # Czynniki sektorowe
    res_factor = obs[len(r):]  # Czynniki resztowe

    # Obliczanie zmiennej kontrolnej dla indywidualnych strat
    control_variate = (
            r[0]**0.5 * m_factor
            + (sec_loading - r[0])**0.5 * sec_factor
            + (1 - sec_loading)**0.5 * res_factor
    )

    # Indeks dla obserwacji poniżej progu
    ind = control_variate < datat
    loss = np.zeros(len(data))

    # Ograniczenie wartości z rozkładu t-Studenta
    if np.any(ind):
        loss[ind] = data.loc[ind, 'm'].values + data.loc[ind, 'd'].values * np.clip(t.rvs(df=3, size=sum(ind)), -5, 5)

    # Zwróć sumę strat i wariancję strat
    return np.sum(loss), np.var(loss)

# Generowanie próbek antetycznych
factors = np.random.normal(0, 1, (num_obvs, len(r) + len(data)))
antithetic_factors = -factors
combined_factors = np.vstack([factors, antithetic_factors])

# Równoległe przetwarzanie próbek
answers = Parallel(n_jobs=-1, verbose=5)(delayed(process_obs)(obs) for obs in combined_factors)
answers = np.array(answers)

# Wyniki
sample_losses = answers[:, 0]
sample_vars = answers[:, 1]

# Obliczenie VaR dla antetycznych próbek
VaR_antithetic = sorted(sample_losses)[int(0.001*num_obvs)]

# Wyniki końcowe
print("Średnia wariancji strat:", np.mean(sample_vars))
print("VaR antetyczny (100-ty wynik):", VaR_antithetic)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 512 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 6112 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 16480 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 29152 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done 44128 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done 59488 tasks      | elapsed:   12.4s


Średnia wariancji strat: 1361.5702719671224
VaR antetyczny (100-ty wynik): -26383.838126145893


[Parallel(n_jobs=-1)]: Done 60000 out of 60000 | elapsed:   12.6s finished


In [11]:
losses = np.array(sample_losses)

In [12]:
quantile_99_9= np.percentile(losses, 0.001)
extreme_losses = losses[losses<quantile_99_9]
np.median(extreme_losses)

-43573.52068710552

In [13]:
import numpy as np
import pandas as pd
from scipy.stats import t, norm, qmc
from joblib import Parallel, delayed

# Wczytanie danych
data = pd.read_csv('data.csv')

# Parametry symulacji
num_obvs = 30_000
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
sector_indices = data['sector'].values
sec_loading = r[sector_indices]
datat = norm.ppf(data['p'])

# Funkcja obliczająca straty dla jednej obserwacji
def process_obs(obs):
    m_factor = obs[0]  # Czynnik ogólny
    sec_factor = obs[:len(r)][sector_indices]  # Czynniki sektorowe
    res_factor = obs[len(r):]  # Czynniki resztowe

    # Obliczanie zmiennej kontrolnej dla indywidualnych strat
    control_variate = (
            r[0]**0.5 * m_factor
            + (sec_loading - r[0])**0.5 * sec_factor
            + (1 - sec_loading)**0.5 * res_factor
    )

    # Indeks dla obserwacji poniżej progu
    ind = control_variate < datat
    loss = np.zeros(len(data))

    # Ograniczenie wartości z rozkładu t-Studenta
    if np.any(ind):
        loss[ind] = data.loc[ind, 'm'].values + data.loc[ind, 'd'].values * np.clip(t.rvs(df=3, size=sum(ind)), -5, 5)

    # Zwróć sumę strat i wariancję strat
    return np.sum(loss), np.var(loss)

# Generowanie próbek za pomocą sekwencji Sobola
def generate_sobol_factors(num_samples, dim):
    sampler = qmc.Sobol(d=dim, scramble=True)
    sobol_samples = sampler.random_base2(m=int(np.log2(num_samples)))

    # Konwersja próbek Sobola na rozkład normalny
    sobol_factors = norm.ppf(sobol_samples)
    return sobol_factors

# Generowanie quasi-Monte Carlo próbek
sobol_factors = generate_sobol_factors(num_obvs, len(r) + len(data))

# Generowanie antetycznych próbek
antithetic_factors = -sobol_factors
combined_factors = np.vstack([sobol_factors, antithetic_factors])

# Równoległe przetwarzanie próbek
answers = Parallel(n_jobs=-1, verbose=5)(delayed(process_obs)(obs) for obs in combined_factors)
answers = np.array(answers)

# Wyniki
sample_losses = answers[:, 0]
sample_vars = answers[:, 1]

# Obliczenie VaR dla antetycznych próbek
VaR_antithetic = sorted(sample_losses)[int(0.001*num_obvs)]

# Wyniki końcowe
print("Średnia wariancji strat:", np.mean(sample_vars))
print("VaR antetyczny (100-ty wynik):", VaR_antithetic)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 512 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 6112 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 16480 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 29152 tasks      | elapsed:    7.1s


Średnia wariancji strat: 1349.8830658967622
VaR antetyczny (100-ty wynik): -24649.74392752423


[Parallel(n_jobs=-1)]: Done 32768 out of 32768 | elapsed:    7.9s finished


In [14]:
import numpy as np
import pandas as pd
from scipy.stats import t, norm
from joblib import Parallel, delayed

# Wczytanie danych
data = pd.read_csv('data.csv')

# Parametry symulacji
num_obvs = 30_000
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
sector_indices = data['sector'].values
sec_loading = r[sector_indices]
datat = norm.ppf(data['p'])

# Wartość oczekiwana dla uproszczonej zmiennej kontrolnej
expected_loss = np.mean(data['m'])

# Funkcja obliczająca straty z użyciem zmiennych kontrolnych
def process_obs_with_cv(obs):
    m_factor = obs[0]  # Czynnik ogólny
    sec_factor = obs[:len(r)][sector_indices]  # Czynniki sektorowe
    res_factor = obs[len(r):]  # Czynniki resztowe

    # Obliczanie zmiennej kontrolnej
    control_variate = (
            r[0]**0.5 * m_factor
            + (sec_loading - r[0])**0.5 * sec_factor
            + (1 - sec_loading)**0.5 * res_factor
    )

    # Indeks dla obserwacji poniżej progu
    ind = control_variate < datat
    loss = np.zeros(len(data))

    # Obliczenie strat i zmiennej kontrolnej
    if np.any(ind):
        loss[ind] = data.loc[ind, 'm'].values + data.loc[ind, 'd'].values * np.clip(t.rvs(df=3, size=sum(ind)), -5, 5)
    raw_loss = np.sum(loss)

    # Dodanie zmiennej kontrolnej (wzór redukujący wariancję)
    cv_adjustment = (np.sum(control_variate) / len(control_variate)) - expected_loss
    adjusted_loss = raw_loss - cv_adjustment

    # Zwróć stratę i jej wariancję
    return adjusted_loss, np.var(loss)

# Generowanie próbek Monte Carlo z antetycznymi próbkami
factors = np.random.normal(0, 1, (num_obvs, len(r) + len(data)))
antithetic_factors = -factors
combined_factors = np.vstack([factors, antithetic_factors])

# Równoległe przetwarzanie próbek
answers = Parallel(n_jobs=-1, verbose=5)(delayed(process_obs_with_cv)(obs) for obs in combined_factors)
answers = np.array(answers)

# Wyniki
sample_losses = answers[:, 0]
sample_vars = answers[:, 1]

# Obliczenie VaR
VaR_antithetic = sorted(sample_losses)[int(0.001*num_obvs)]

# Wyniki końcowe
print("Średnia wariancji strat:", np.mean(sample_vars))
print("VaR antetyczny (100-ty wynik):", VaR_antithetic)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 512 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 6112 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 16480 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 29152 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done 44128 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 59488 tasks      | elapsed:   12.5s


Średnia wariancji strat: 1335.481869102711
VaR antetyczny (100-ty wynik): -25925.59041319705


[Parallel(n_jobs=-1)]: Done 60000 out of 60000 | elapsed:   12.7s finished


In [15]:
import numpy as np
import pandas as pd
from scipy.stats import t, norm
from joblib import Parallel, delayed

# Wczytanie danych
data = pd.read_csv('data.csv')

# Parametry symulacji
num_obvs = 30_000  # Liczba obserwacji
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
sector_indices = data['sector'].values
sec_loading = r[sector_indices]
datat = norm.ppf(data['p'])

# Funkcja obliczająca straty dla jednej próby
def process_obs(obs):
    # Wybór czynników (ogólny, sektorowy, resztowy)
    m_factor = obs[0]
    sec_factor = obs[:len(r)][sector_indices]
    res_factor = obs[len(r):]

    # Obliczenie indywidualnej zmiennej losowej
    control_variate = (
            r[0]**0.5 * m_factor
            + (sec_loading - r[0])**0.5 * sec_factor
            + (1 - sec_loading)**0.5 * res_factor
    )
    # Sprawdzenie progu
    ind = control_variate < datat
    loss = np.zeros(len(data))

    # Generowanie strat dla spełnionych warunków
    if np.any(ind):
        loss[ind] = (
                data.loc[ind, 'm'].values
                + data.loc[ind, 'd'].values * np.clip(t.rvs(df=3, size=sum(ind)), -5, 5)
        )
    return np.sum(loss), np.var(loss)

# Generowanie antetycznych próbek
dim = len(r) + len(data)
factors = np.random.normal(0, 1, (num_obvs, dim))
antithetic_factors = -factors
combined_factors = np.vstack([factors, antithetic_factors])

# Równoległe przetwarzanie próbek
print("Rozpoczęcie obliczeń...")
answers = Parallel(n_jobs=-1, verbose=10)(
    delayed(process_obs)(obs) for obs in combined_factors
)
answers = np.array(answers)

# Wyniki
sample_losses = answers[:, 0]
sample_vars = answers[:, 1]

# Obliczenie VaR
VaR_antithetic = sorted(sample_losses)[int(0.001*num_obvs)]

# Wyświetlenie wyników
print("Średnia wariancji strat:", np.mean(sample_vars))
print("VaR antetyczny (100-ty wynik):", VaR_antithetic)


Rozpoczęcie obliczeń...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0091s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0676s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0631s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 164 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 224 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0799s.) Setting batch_size=16.
[Parallel(n_jobs=-1)]: Done 360 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Batch com

Średnia wariancji strat: 1362.583623111245
VaR antetyczny (100-ty wynik): -26211.276589941175


[Parallel(n_jobs=-1)]: Done 60000 out of 60000 | elapsed:   12.8s finished


In [16]:
import numpy as np
import pandas as pd
from scipy.stats import t, norm
from joblib import Parallel, delayed

# Wczytanie danych
data = pd.read_csv('data.csv')

# Parametry symulacji
num_obvs = 15_000  # Zmniejszona liczba prób
num_bootstraps = 5  # Liczba powtórzeń bootstrap
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
sector_indices = data['sector'].values
sec_loading = r[sector_indices]
datat = norm.ppf(data['p'])

# Funkcja obliczająca straty dla jednej próby
def process_obs(obs):
    m_factor = obs[0]
    sec_factor = obs[:len(r)][sector_indices]
    res_factor = obs[len(r):]

    control_variate = (
            r[0]**0.5 * m_factor
            + (sec_loading - r[0])**0.5 * sec_factor
            + (1 - sec_loading)**0.5 * res_factor
    )
    ind = control_variate < datat
    loss = np.zeros(len(data))

    if np.any(ind):
        loss[ind] = (
                data.loc[ind, 'm'].values
                + data.loc[ind, 'd'].values * np.clip(t.rvs(df=3, size=sum(ind)), -5, 5)
        )
    return np.sum(loss), np.var(loss)

# Generowanie antetycznych próbek
def generate_factors(num_obvs, dim):
    factors = np.random.normal(0, 1, (num_obvs, dim))
    antithetic_factors = -factors
    return np.vstack([factors, antithetic_factors])

# Funkcja bootstrapująca
def bootstrap_run():
    combined_factors = generate_factors(num_obvs, len(r) + len(data))
    answers = Parallel(n_jobs=-1)(
        delayed(process_obs)(obs) for obs in combined_factors
    )
    answers = np.array(answers)
    return answers[:, 0], answers[:, 1]

# Główna pętla bootstrap
all_losses, all_vars = [], []
for i in range(num_bootstraps):
    print(f"Bootstrap iteracja {i+1}/{num_bootstraps}...")
    losses, vars_ = bootstrap_run()
    all_losses.append(losses)
    all_vars.append(vars_)

# Agregowanie wyników
all_losses = np.vstack(all_losses)
all_vars = np.vstack(all_vars)

mean_losses = np.mean(all_losses, axis=0)
mean_vars = np.mean(all_vars, axis=0)

# Obliczenie VaR
VaR_antithetic = sorted(mean_losses)[int(0.001*num_obvs)]

# Wyświetlenie wyników
print("Średnia wariancji strat:", np.mean(mean_vars))
print("VaR antetyczny (100-ty wynik):", VaR_antithetic)


Bootstrap iteracja 1/5...
Bootstrap iteracja 2/5...
Bootstrap iteracja 3/5...
Bootstrap iteracja 4/5...
Bootstrap iteracja 5/5...
Średnia wariancji strat: 1355.7348658528772
VaR antetyczny (100-ty wynik): -8304.806069612436


In [17]:
import numpy as np
import pandas as pd
from scipy.stats import t, norm
from joblib import Parallel, delayed

# Wczytanie danych
data = pd.read_csv('data.csv')

# Parametry symulacji
num_obvs = 30_000
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
sector_indices = data['sector'].values
sec_loading = r[sector_indices]
datat = norm.ppf(data['p'])

# Funkcja Importance Sampling
def process_obs_IS(obs, bias):
    # Dodanie biasu do próbek
    obs_shifted = obs + bias

    # Wybór czynników (ogólny, sektorowy, resztowy)
    m_factor = obs_shifted[0]
    sec_factor = obs_shifted[:len(r)][sector_indices]
    res_factor = obs_shifted[len(r):]

    # Obliczenie zmiennej losowej i strat
    control_variate = (
            r[0]**0.5 * m_factor
            + (sec_loading - r[0])**0.5 * sec_factor
            + (1 - sec_loading)**0.5 * res_factor
    )
    ind = control_variate < datat
    loss = np.zeros(len(data))

    if np.any(ind):
        loss[ind] = (
                data.loc[ind, 'm'].values
                + data.loc[ind, 'd'].values * np.clip(t.rvs(df=3, size=sum(ind)), -5, 5)
        )
    # Korekcja wag (Importance Sampling)
    weight = np.exp(-0.5 * np.sum(bias**2) + 0.5 * np.sum(obs**2))
    weighted_loss = np.sum(loss) * weight

    return weighted_loss, np.var(loss)

# Generowanie próbek z Importance Sampling
dim = len(r) + len(data)
bias = np.ones(dim) * 0.5  # Wektor biasu przesuwający rozkład

factors = np.random.normal(0, 1, (num_obvs, dim))
antithetic_factors = -factors
combined_factors = np.vstack([factors, antithetic_factors])

# Równoległe przetwarzanie próbek
print("Rozpoczęcie Importance Sampling...")
answers = Parallel(n_jobs=-1, verbose=10)(
    delayed(process_obs_IS)(obs, bias) for obs in combined_factors
)
answers = np.array(answers)

# Wyniki
sample_losses = answers[:, 0]
sample_vars = answers[:, 1]

# Obliczenie VaR
VaR_IS = sorted(sample_losses)[int(0.001*num_obvs)]

# Wyświetlenie wyników
print("Średnia wariancji strat:", np.mean(sample_vars))
print("VaR z Importance Sampling (100-ty wynik):", VaR_IS)


Rozpoczęcie Importance Sampling...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0212s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0589s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0660s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 164 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 224 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0750s.) Setting batch_size=16.
[Parallel(n_jobs=-1)]: Done 360 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Batch com

Średnia wariancji strat: 94.53512777298837
VaR z Importance Sampling (100-ty wynik): inf


In [18]:
import numpy as np
import pandas as pd
from scipy.stats import t, norm
from joblib import Parallel, delayed

# Wczytanie danych
data = pd.read_csv('data.csv')

# Parametry symulacji
num_obvs = 30_000  # Zwiększona liczba prób
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
sector_indices = data['sector'].values
sec_loading = r[sector_indices]
datat = norm.ppf(data['p'])

# Funkcja obliczająca straty z mniejszymi stopniami swobody (więcej ciężkich ogonów)
def process_obs_stable(obs):
    m_factor = obs[0]
    sec_factor = obs[:len(r)][sector_indices]
    res_factor = obs[len(r):]

    # Obliczenie zmiennej losowej z t-Studenta
    control_variate = (
            r[0]**0.5 * m_factor
            + (sec_loading - r[0])**0.5 * sec_factor
            + (1 - sec_loading)**0.5 * res_factor
    )

    # Zmiana wartości granicznych
    ind = control_variate < datat
    loss = np.zeros(len(data))

    if np.any(ind):
        # Zastosowanie rozkładu t-Studenta z mniejszymi stopniami swobody dla większej odpornosci na ekstremalne straty
        loss[ind] = (
                data.loc[ind, 'm'].values
                + data.loc[ind, 'd'].values * np.clip(t.rvs(df=4, size=sum(ind)), -10, 10)
        )
    return np.sum(loss), np.var(loss)

# Generowanie próbek i antetycznych próbek
dim = len(r) + len(data)
factors = np.random.normal(0, 1, (num_obvs, dim))
antithetic_factors = -factors
combined_factors = np.vstack([factors, antithetic_factors])

# Równoległe przetwarzanie próbek
print("Rozpoczęcie obliczeń z nowym rozkładem t-Studenta...")
answers = Parallel(n_jobs=-1, verbose=10)(
    delayed(process_obs_stable)(obs) for obs in combined_factors
)
answers = np.array(answers)

# Wyniki
sample_losses = answers[:, 0]
sample_vars = answers[:, 1]

# Obliczenie VaR
VaR_stable = sorted(sample_losses)[int(0.001*num_obvs)]

# Wyświetlenie wyników
print("Średnia wariancji strat:", np.mean(sample_vars))
print("VaR z t-Studenta (100-ty wynik):", VaR_stable)


Rozpoczęcie obliczeń z nowym rozkładem t-Studenta...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0080s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0630s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0719s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 164 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 224 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0967s.) Setting batch_size=16.
[Parallel(n_jobs=-1)]: Done 360 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Batch com

Średnia wariancji strat: 1355.052253839234
VaR z t-Studenta (100-ty wynik): -27808.47321504282


[Parallel(n_jobs=-1)]: Done 60000 out of 60000 | elapsed:   13.3s finished


In [19]:
import numpy as np
import pandas as pd
from scipy.stats import norm, t
from joblib import Parallel, delayed

# Wczytanie danych
data = pd.read_csv('data.csv')

# Parametry symulacji
num_obvs = 30_000  # Liczba prób Monte Carlo
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
sector_indices = data['sector'].values
sec_loading = r[sector_indices]
datat = norm.ppf(data['p'])

# Funkcja obliczająca straty z ważeniem
def process_obs_weighted(obs, weight_factor):
    # Modyfikacja rozkładu próbek przez skalowanie wagami
    m_factor = obs[0] * weight_factor
    sec_factor = obs[:len(r)][sector_indices] * weight_factor
    res_factor = obs[len(r):] * weight_factor

    # Obliczenie zmiennej losowej
    control_variate = (
            r[0]**0.5 * m_factor
            + (sec_loading - r[0])**0.5 * sec_factor
            + (1 - sec_loading)**0.5 * res_factor
    )

    # Graniczny próg
    ind = control_variate < datat
    loss = np.zeros(len(data))

    if np.any(ind):
        # Zastosowanie rozkładu t-Studenta dla resztowych strat
        loss[ind] = (
                data.loc[ind, 'm'].values
                + data.loc[ind, 'd'].values * np.clip(t.rvs(df=3, size=sum(ind)), -5, 5)
        )
    return np.sum(loss), np.var(loss)

# Funkcja dla obliczania wag (ważenie obszarów)
def calculate_weights(factors):
    # Proste ważenie bazujące na rozkładzie normalnym, które koncentruje się na dużych stratach
    weights = np.exp(-np.abs(factors) / np.std(factors))
    return weights

# Generowanie próbek
dim = len(r) + len(data)
factors = np.random.normal(0, 1, (num_obvs, dim))

# Obliczanie wag dla próbek
weights = calculate_weights(factors)

# Generowanie próbek antytetycznych
antithetic_factors = -factors
combined_factors = np.vstack([factors, antithetic_factors])

# Równoległe przetwarzanie próbek
answers = Parallel(n_jobs=-1, verbose=10)(
    delayed(process_obs_weighted)(obs, weight_factor)
    for obs, weight_factor in zip(combined_factors, weights)
)
answers = np.array(answers)

# Wyniki
sample_losses = answers[:, 0]
sample_vars = answers[:, 1]

# Obliczenie VaR
VaR_weighted = sorted(sample_losses)[int(0.001*num_obvs)]

# Wyświetlenie wyników
print("Średnia wariancji strat:", np.mean(sample_vars))
print("VaR ważony (100-ty wynik):", VaR_weighted)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0250s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0010s.) Setting batch_size=4.


ValueError: operands could not be broadcast together with shapes (5658,) (5666,) 

In [20]:
import numpy as np
import pandas as pd
from scipy.stats import norm, t
from joblib import Parallel, delayed

# Wczytanie danych
data = pd.read_csv('data.csv')

# Parametry symulacji
num_obvs = 30_000  # Liczba prób Monte Carlo
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
sector_indices = data['sector'].values
sec_loading = r[sector_indices]
datat = norm.ppf(data['p'])

# Funkcja do podziału próbek na warstwy
def stratified_sampling(num_obvs, num_strata):
    # Tworzenie warstw: w tym przypadku losowe podzielenie próbek
    strata = np.linspace(-4, 4, num_strata)  # Rozkład warstw w przestrzeni
    samples = []

    for i in range(num_strata):
        strata_samples = np.random.normal(strata[i], 1, num_obvs // num_strata)
        samples.append(strata_samples)

    return np.concatenate(samples)

# Funkcja obliczająca straty z antytezą
def process_obs_antithetic(obs):
    # Użycie próbek antytetycznych
    m_factor = obs[0]
    sec_factor = obs[:len(r)][sector_indices]
    res_factor = obs[len(r):]

    control_variate = (
            r[0]**0.5 * m_factor
            + (sec_loading - r[0])**0.5 * sec_factor
            + (1 - sec_loading)**0.5 * res_factor
    )

    # Graniczny próg
    ind = control_variate < datat
    loss = np.zeros(len(data))

    if np.any(ind):
        # Zastosowanie rozkładu t-Studenta dla resztowych strat
        loss[ind] = (
                data.loc[ind, 'm'].values
                + data.loc[ind, 'd'].values * np.clip(t.rvs(df=3, size=sum(ind)), -5, 5)
        )
    return np.sum(loss), np.var(loss)

# Generowanie próbek z stratified sampling
num_strata = 10  # Liczba warstw, można dostosować
stratified_factors = stratified_sampling(num_obvs, num_strata)

# Generowanie próbek antytetycznych
antithetic_factors = -stratified_factors
combined_factors = np.vstack([stratified_factors, antithetic_factors])

# Równoległe przetwarzanie próbek
answers = Parallel(n_jobs=-1, verbose=10)(
    delayed(process_obs_antithetic)(obs) for obs in combined_factors
)
answers = np.array(answers)

# Wyniki
sample_losses = answers[:, 0]
sample_vars = answers[:, 1]

# Obliczenie VaR
VaR_stratified = sorted(sample_losses)[int(0.001*num_obvs)]

# Wyświetlenie wyników
print("Średnia wariancji strat:", np.mean(sample_vars))
print("VaR stratified (100-ty wynik):", VaR_stratified)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    3.6s remaining:    0.0s


ValueError: operands could not be broadcast together with shapes (5658,) (29992,) 

In [21]:
import numpy as np
import pandas as pd
from scipy.stats import t, norm
from joblib import Parallel, delayed
from scipy.stats import qmc

# Wczytanie danych
data = pd.read_csv('data.csv')

# Parametry symulacji
num_obvs = 30_000  # Zwiększona liczba prób
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
sector_indices = data['sector'].values
sec_loading = r[sector_indices]
datat = norm.ppf(data['p'])

# Funkcja obliczająca straty
def process_obs_qmc(obs):
    m_factor = obs[0]
    sec_factor = obs[:len(r)][sector_indices]
    res_factor = obs[len(r):]

    # Obliczenie zmiennej losowej
    control_variate = (
            r[0]**0.5 * m_factor
            + (sec_loading - r[0])**0.5 * sec_factor
            + (1 - sec_loading)**0.5 * res_factor
    )

    # Indeksy strat
    ind = control_variate < datat
    loss = np.zeros(len(data))

    if np.any(ind):
        loss[ind] = (
                data.loc[ind, 'm'].values
                + data.loc[ind, 'd'].values * np.clip(t.rvs(df=3, size=sum(ind)), -5, 5)
        )
    return np.sum(loss), np.var(loss)

# Generowanie próbek z użyciem punktów Sobola
sampler = qmc.Sobol(d=len(r) + len(data), scramble=True)
factors = sampler.random(num_obvs)  # Generujemy próbki Sobola
factors = 2 * factors - 1  # Transformacja na rozkład jednostajny [-1, 1]

# Zastosowanie antetycznych próbek
antithetic_factors = -factors
combined_factors = np.vstack([factors, antithetic_factors])

# Równoległe przetwarzanie próbek
print("Rozpoczęcie obliczeń z metodą Quasi-Monte Carlo...")
answers = Parallel(n_jobs=-1, verbose=10)(
    delayed(process_obs_qmc)(obs) for obs in combined_factors
)
answers = np.array(answers)

# Wyniki
sample_losses = answers[:, 0]
sample_vars = answers[:, 1]

# Obliczenie VaR
VaR_qmc = sorted(sample_losses)[int(0.001*num_obvs)]

# Wyświetlenie wyników
print("Średnia wariancji strat:", np.mean(sample_vars))
print("VaR z QMC (100-ty wynik):", VaR_qmc)


D:\ProgramData\anaconda3\Lib\site-packages\scipy\stats\_qmc.py:804: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  sample = self._random(n, workers=workers)


Rozpoczęcie obliczeń z metodą Quasi-Monte Carlo...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1894s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0650s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0770s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 244 tasks      | elapsed: 

Średnia wariancji strat: 0.0
VaR z QMC (100-ty wynik): 0.0


[Parallel(n_jobs=-1)]: Done 59814 tasks      | elapsed:   16.8s
[Parallel(n_jobs=-1)]: Done 60000 out of 60000 | elapsed:   16.9s finished


In [22]:
import numpy as np
import pandas as pd
from scipy.stats import t, norm
from joblib import Parallel, delayed
from scipy.stats import qmc

# Wczytanie danych
data = pd.read_csv('data.csv')

# Parametry symulacji
num_obvs = 30_000  # Zwiększona liczba prób
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
sector_indices = data['sector'].values  # Indeksy sektorów w danych
sec_loading = r[sector_indices]  # Ładowanie ryzyka po sektorach
datat = norm.ppf(data['p'])  # Zmienna odpowiadająca wartościom prawdopodobieństwa

# Funkcja obliczająca straty
def process_obs_qmc(obs):
    m_factor = obs[0]
    sec_factor = obs[1:len(r)+1]  # Wartości dla każdego sektora
    res_factor = obs[len(r)+1:]  # Reszta czynników

    # Obliczenie zmiennej losowej
    control_variate = (
            r[0]**0.5 * m_factor
            + (sec_loading - r[0])**0.5 * sec_factor
            + (1 - sec_loading)**0.5 * res_factor
    )

    # Indeksy strat
    ind = control_variate < datat
    loss = np.zeros(len(data))

    # Jeśli są jakieś straty (ind != 0)
    if np.any(ind):
        loss[ind] = (
                data.loc[ind, 'm'].values
                + data.loc[ind, 'd'].values * np.clip(t.rvs(df=3, size=sum(ind)), -5, 5)
        )
    return np.sum(loss), np.var(loss)

# Generowanie próbek z użyciem punktów Sobola
sampler = qmc.Sobol(d=len(r) + len(data), scramble=True)
factors = sampler.random(num_obvs)  # Generujemy próbki Sobola
factors = 2 * factors - 1  # Transformacja na rozkład jednostajny [-1, 1]

# Zastosowanie antetycznych próbek
antithetic_factors = -factors
combined_factors = np.vstack([factors, antithetic_factors])

# Równoległe przetwarzanie próbek
print("Rozpoczęcie obliczeń z metodą Quasi-Monte Carlo...")
answers = Parallel(n_jobs=-1, verbose=10)(
    delayed(process_obs_qmc)(obs) for obs in combined_factors
)
answers = np.array(answers)

# Wyniki
sample_losses = answers[:, 0]
sample_vars = answers[:, 1]

# Obliczenie VaR
VaR_qmc = sorted(sample_losses)[int(0.001*num_obvs)]  # Wyciąganie 100-tego wyniku z posortowanych strat

# Wyświetlenie wyników
print("Średnia wariancji strat:", np.mean(sample_vars))
print("VaR z QMC (100-ty wynik):", VaR_qmc)


D:\ProgramData\anaconda3\Lib\site-packages\scipy\stats\_qmc.py:804: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  sample = self._random(n, workers=workers)


Rozpoczęcie obliczeń z metodą Quasi-Monte Carlo...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1090s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0070s.) Setting batch_size=4.


ValueError: operands could not be broadcast together with shapes (5658,) (8,) 

BASELINE

In [23]:
import numpy as np, pandas as pd, scipy.stats as st
from scipy.stats import norm
from joblib import Parallel, delayed
from scipy.stats import qmc

num_obvs = 30_000
data = pd.read_csv("data.csv")
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
data["sec_loading"], data["t"] = r[data["sector"].values], st.norm.ppf(data.p)
factors, sample = np.random.normal(0,1, (num_obvs, len(r)+len(data))), []

def process_obs(obs):
    m_factor, sec_factor, res_factor = obs[0], obs[:len(r)][data.sector.values], obs[len(r):]
    ind = (
            r[0]**.5 * m_factor + (data.sec_loading-r[0])**.5 * sec_factor + (1-data.sec_loading)**.5 * res_factor
            < data.t
    )
    loss = np.zeros((len(data),))
    loss[ind] = data[ind].m + data[ind].d * np.random.standard_t(3, size=sum(ind))
    return sum(loss), np.var(loss)

answers = Parallel(n_jobs=-1)(delayed(process_obs)(obs) for obs in factors)
answers = np.array(answers)

# Wyniki
sample_losses = answers[:, 0]
sample_vars = answers[:, 1]

# Obliczenie VaR
VaR_qmc = sorted([-s for s in sample_losses])[int(0.001*num_obvs)]  # Wyciąganie 100-tego wyniku z posortowanych strat


In [24]:
VaR_qmc

-17227.451262458308

In [25]:
np.mean(sample_vars)

1368.5773664035662

In [26]:
from scipy.stats import qmc, norm
from joblib import Parallel, delayed
import numpy as np
import pandas as pd
import scipy.stats as st
from scipy.stats import t

num_obvs = 30_000
data = pd.read_csv("data.csv")
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
data["sec_loading"], data["t"] = r[data["sector"].values], st.norm.ppf(data.p)
factors, sample = np.random.normal(0,1, (num_obvs, len(r)+len(data))), []

# Function to process a single observation
def process_obs(obs):
    m_factor = obs[0]
    sec_factor = obs[:len(r)][data.sector.values]
    res_factor = obs[len(r):]

    ind = (
                  r[0]**0.5 * m_factor
                  + (data.sec_loading - r[0])**0.5 * sec_factor
                  + (1 - data.sec_loading)**0.5 * res_factor
          ) < data.t

    loss = np.zeros(len(data))
    loss[ind] = data[ind].m + data[ind].d * t.rvs(df=3, size=sum(ind))
    return sum(loss), np.var(loss)

# Generate LHS factors
def generate_lhs_factors(num_obvs, num_dimensions):
    # Create Latin Hypercube Sampler
    lhs_sampler = qmc.LatinHypercube(d=num_dimensions)

    # Generate LHS samples in [0, 1]
    lhs_samples = lhs_sampler.random(n=num_obvs)

    # Transform to standard normal distribution
    lhs_factors = norm.ppf(lhs_samples)

    return lhs_factors

# Run LHS method
def run_lhs(data, r, num_obvs=30_000):
    num_dimensions = len(r) + len(data)  # Total risk factors
    lhs_factors = generate_lhs_factors(num_obvs, num_dimensions)  # Generate LHS samples

    # Parallelized loss computation
    answers = Parallel(n_jobs=-1, verbose=10)(
        delayed(process_obs)(obs) for obs in lhs_factors
    )
    answers = np.array(answers)
    
    sample = answers[:, 0]
    vars = answers[:, 1]

    # Compute VaR (99.9% quantile)
    VaR = np.percentile(np.array(sample), 99.9)
    var = np.mean(vars)
    
    return VaR, var

# Simulation
num_sims = [100000]
lhs_results = [run_lhs(data, r, num_obvs=n) for n in num_sims]

# Output results
print(lhs_results)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1885s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0670s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 148 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0851s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done 216 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1

[(17624.812468017866, 1353.577483202275)]
